In [1]:
import os
import pandas as pd
import requests
import time
from dotenv import load_dotenv


load_dotenv()
SLACK_WEBHOOK_URL = os.getenv("SLACK_WEBHOOK_URL")

#  Function to send Slack alerts
def send_slack_alert(message: str):
    if not SLACK_WEBHOOK_URL:
        print("⚠️ No Slack webhook URL set.")
        return
    try:
        response = requests.post(SLACK_WEBHOOK_URL, json={"text": message}, timeout=10)
        if response.status_code == 200:
            print("✅ Alert sent successfully.")
        else:
            print(f"❌ Failed to send alert: {response.status_code} {response.text}")
    except Exception as e:
        print(f"❌ Error sending alert: {e}")

In [2]:
def main():
    # Load the CSV file
    df = pd.read_csv("ai_news_with_sentiment_scores.csv", encoding="utf-8")

    # Filter only negative news
    negatives = df[df["sentiment"].str.lower() == "negative"]
    negatives = negatives.sort_values(by="sentiment_score") #sorting to get highest values
    message_count = 0

    if negatives.empty:
        print("No negative news found.")
        return

    # Send one Slack message per negative article
    for i, row in negatives.iterrows():
        title = row.get("title", "Untitled")
        url = row.get("url", "")
        score = row.get("sentiment_score", "N/A")
        sentiment = row.get("sentiment", "Unknown")

        message = (
            f"*Negative News Alert!*\n"
            f"*Title:* {title}\n"
            f"*Sentiment:* {sentiment} ({score})\n"
            f"*URL:* {url}"
        )

        send_slack_alert(message)
        time.sleep(1) 

        message_count += 1
        if message_count == 10: #sending only selected messages
            break

if __name__ == "__main__":
    main()

✅ Alert sent successfully.
✅ Alert sent successfully.
✅ Alert sent successfully.
✅ Alert sent successfully.
✅ Alert sent successfully.
✅ Alert sent successfully.
✅ Alert sent successfully.
✅ Alert sent successfully.
✅ Alert sent successfully.
✅ Alert sent successfully.
